# Data C200 Final Project

This is the Jupyter Notebook file of Data C200. Data in the `data` folder.

## Step 0 : Module and Data Import

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import geopandas as gpd # For geometry data precessing
import geoplot as gplt # For mapping projections onto US map
import geoplot.crs as gcrs # coordinate reference system

import folium
import folium.plugins # The Folium Javascript Map Library

import os 
import re

%matplotlib inline

pd.set_option('display.max_rows', 20)

In [2]:
drd_filename = os.path.join('data', 'Accidental_Drug_Related_Deaths_2012-2018.csv')

drug_related_deaths = pd.read_csv(drd_filename)
drug_related_deaths.head()

,ID,Date,DateType,Age,Sex,Race,ResidenceCity,ResidenceCounty,ResidenceState,DeathCity,...,Tramad,Morphine_NotHeroin,Hydromorphone,Other,OpiateNOS,AnyOpioid,MannerofDeath,DeathCityGeo,ResidenceCityGeo,InjuryCityGeo
0,14-0273,06/28/2014 12:00:00 AM,DateReported,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Accident,"CT\n(41.575155, -72.738288)","CT\n(41.575155, -72.738288)","CT\n(41.575155, -72.738288)"
1,13-0102,03/21/2013 12:00:00 AM,DateofDeath,48.0,Male,Black,NORWALK,NaN,NaN,NORWALK,...,NaN,NaN,NaN,NaN,NaN,NaN,Accident,"Norwalk, CT\n(41.11805, -73.412906)","NORWALK, CT\n(41.11805, -73.412906)","CT\n(41.575155, -72.738288)"
2,16-0165,03/13/2016 12:00:00 AM,DateofDeath,30.0,Female,White,SANDY HOOK,FAIRFIELD,CT,DANBURY,...,NaN,NaN,NaN,NaN,NaN,Y,Accident,"Danbury, CT\n(41.393666, -73.451539)","SANDY HOOK, CT\n(41.419998, -73.282501)",NaN
3,16-0208,03/31/2016 12:00:00 AM,DateofDeath,23.0,Male,White,RYE,WESTCHESTER,NY,GREENWICH,...,NaN,NaN,NaN,NaN,NaN,Y,Accident,"Greenwich, CT\n(41.026526, -73.628549)",NaN,NaN
4,13-0052,02/13/2013 12:00:00 AM,DateofDeath,22.0,Male,"Asian, Other",FLUSHING,QUEENS,NaN,GREENWICH,...,NaN,NaN,NaN,NaN,NaN,NaN,Accident,"Greenwich, CT\n(41.026526, -73.628549)",NaN,"CT\n(41.575155, -72.738288)"


In [3]:
mnc_filename = os.path.join('data', 'mnc_ct.csv')

mnc_ct = pd.read_csv(mnc_filename)
mnc_ct.head()

,Number,Town,Designation,Dateestablished,Land area(square miles),Population(in 2010),Form ofgovernment,County,Council of Governments,Native Americanname
0,1,Andover,Town,1848,15.46,"3,303",Town meeting,Tolland County,Capitol Region,NaN
1,2,Ansonia,City,1889,6.03,"19,249",Mayor-council,New Haven County,Naugatuck Valley,NaN
2,3,Ashford,Town,1714,38.79,"4,100",Town meeting,Windham County,Northeast CT,NaN
3,4,Avon,Town,1830,23.12,"18,098",Council-manager,Hartford County,Capitol Region,NaN
4,5,Barkhamsted,Town,1779,36.22,"3,620",Town meeting,Litchfield County,Northwest Hills,NaN


In [4]:
cdp_filename = os.path.join('data', 'cdp_ct.csv')

cdp_ct = pd.read_csv(cdp_filename)
cdp_ct.head()

,CDP,Population,County
0,Baltic,"1,250",New London
1,Bethel,"9,549",Fairfield
2,Bethlehem Village,"2,021",Litchfield
3,Blue Hills,"2,901",Hartford
4,Branford Center,"5,819",New Haven


In [5]:
br_filename = os.path.join('data', 'borough_ct.csv')

br_ct = pd.read_csv(br_filename)
br_ct.head()

,Borough,Parent town,Date incorporated,Current status
0,Bridgeport,Stratford,1800,Consolidated city-town separate from original ...
1,Stonington,Stonington,1801,Borough
2,Guilford,Guilford,1815,Disincorporated 1941
3,Essex,Deep River (Saybrook),1820,Disincorporated 1854 when Town of Essex was in...
4,Killingworth (Clinton),Killingworth,1820,Disincorporated 1838 when Town of Clinton was ...


## Step 1 : Exploratory Data Analysis (EDA)

### Main Dataset : [drug related deaths](https://www.kaggle.com/muhakabartay/accidental-drug-related-deaths-20122018) (_drd for short_)

In [6]:
drug_related_deaths.describe()

,Age
count,5102.000000
mean,41.964916
std,12.336639
min,14.000000
25%,32.000000
50%,42.000000
75%,52.000000
max,87.000000


In [7]:
# Basic information
drug_related_deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5105 entries, 0 to 5104
Data columns (total 41 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   5105 non-null   object 
 1   Date                 5103 non-null   object 
 2   DateType             5103 non-null   object 
 3   Age                  5102 non-null   float64
 4   Sex                  5099 non-null   object 
 5   Race                 5092 non-null   object 
 6   ResidenceCity        4932 non-null   object 
 7   ResidenceCounty      4308 non-null   object 
 8   ResidenceState       3556 non-null   object 
 9   DeathCity            5100 non-null   object 
 10  DeathCounty          4005 non-null   object 
 11  Location             5081 non-null   object 
 12  LocationifOther      590 non-null    object 
 13  DescriptionofInjury  4325 non-null   object 
 14  InjuryPlace          5039 non-null   object 
 15  InjuryCity           3349 non-null   o

#### Observational EDA for `drug_related_death` DataFrame

1. **File formats and structure**: csv file (rectangular table), 41 columns in total

_Can we divide the 41 columns into different sets (packages) ?_

Yes. It can be divided as followed: 

* Basic information (e.g. age, sex, race, date of death)
* Residence location (city, county, state, and geometry/coordinate) - This would be main focus of data cleaning, such as joining this table with another table (from a different dataset) containing Connecticut city information.
* Death location (city, county, and geometry/coordinate) - This excludes all `state` information because **this dataset only focuses on drug related death in Connecticut**.
* Injury location (city, county, state, and geometry/coordinate) 
* Drug classification (e.g. opioids such like methadone, heroin and fentanyl, alcohol and others)
    
2. **Granularity**: Each reported individual with drug abuse related deaths in Connecticut

3. **Scope**: 

_Does my data cover my area of interest?_

Not entirely. We have some missing values on states and these need to be filled by joining tables.

_Is our dataset too expansive?_

This dataset is expansive in the way that it only covers drug related deaths data rather than all drug use data, which may contribute to misleading conclusions about drug use in general.

_Does our data have the correct time frames?_

Since all data entries are from Connecticut, the time frames will be Eastern Time (UTC-5:00) by default. Therefore, the time frame is uniform.

4. **Temporality**: Time of the day is not reliable in this dataset, since all values of time of the day are set as `12:00:00 AM` for convinience. For timestamps, the main focus would be the date, which is reliable in this dataset.

5. **Faithfulness and Missing Values**: The faithfulness can be further explored by our data cleaning. Missing values are obvious in several columns (especially states) for brute simplicity. Whenever possible, we will try to hypothesize reasons for the existence of certain null values in our dataset (such as for time of death or for city).

    * _Update_ : Some of the missing values that would be problematic for our analysis:
        * County names and state names : many data entries only have city names with non-null values, counties and states remain null. This has to be solved in a way not to destroy some subset of people in our population of interest, but excluded by NaN values from our sampling frame.
        * Geometries : we would like to utilize geometry information in our data analysis. Null values cannot be used in map projections.

#### About location null values

We can notice that many missing values are in location columns (i.e. Residence/Death/Injury locations). In these locations, we put our focus on `DeathCity` column for the following two reasons:

1. **Why death location?** Since this dataset have all entries of drug-related deaths in the State of Connecticut, the location of death reported must be within Connecticut, without cities or towns from other states in America. Hence, the list of cities or towns (i.e. municipalities) in Connecticut can be utilized.

2. **Why death cities?** Between both of the columns `DeathCity` and `DeathCounty`, `DeathCity` has the biggest number of non-null values.

In [8]:
drug_related_deaths['DeathCity'].unique()

array([nan, 'NORWALK', 'DANBURY', 'GREENWICH', 'BRISTOL', 'WEST HARTFORD',
       'NEW HAVEN', 'EAST HARTFORD', 'MONTVILLE', 'STRATFORD', 'HARTFORD',
       'WALLINGFORD', 'MIDDLETOWN', 'WINDHAM', 'BRIDGEPORT', 'DERBY',
       'COS COB', 'MERIDEN', 'ENFIELD', 'TORRINGTON', 'WATERBURY',
       'MASHANTUCKET', 'MILFORD', 'SOMERS', 'STAMFORD', 'NEW BRITAIN',
       'WILLIMANTIC', 'WATERTOWN', 'BALTIC', 'RIDGEFIELD', 'NEW LONDON',
       'BROOKFIELD', 'SHELTON', 'NORWICH', 'NAUGATUCK', 'VERNON',
       'FAIRFIELD', 'EAST HAMPTON', 'SHARON', 'MONROE', 'WINCHESTER',
       'BRANFORD', 'MANCHESTER', 'WOLCOTT', 'WEST SUFFIELD', 'FARMINGTON',
       'NORTH BRANFORD', 'NEW MILFORD', 'STONINGTON', 'PRESTON', 'GROTON',
       'SOUTHINGTON', 'SOUTH WINDSOR', 'JEWETT CITY', 'EAST WINDSOR',
       'OLD SAYBROOK', 'BROAD BROOK', 'PORTLAND', 'HEBRON', 'DARIEN',
       'LEDYARD', 'TRUMBULL', 'BURLINGTON', 'BLOOMFIELD', 'TERRYVILLE',
       'STAFFORD SPRINGS', 'WEST HAVEN', 'THOMPSON', 'OXFORD',
       '

From this array of unique death city names in `drug_related_death`, it implies different types of cities:

1. **Cities or towns (Municipalities)** : As are the most common city names in this list.

2. **[Census-designated places (CDPs)](https://en.wikipedia.org/wiki/List_of_census-designated_places_in_Connecticut)** : Census-designated places (CDPs) are unincorporated communities lacking elected municipal officers and boundaries with legal status. Connecticut has 112 census-designated places. Some CDPs are deprecated former cities (_e.g. Willimantic, _) and do not have separate pages from their parent town, while others are coterminous with their parent town. Further, some CDPs take the name of their parent town.

3. **Villages, historical districts or other communities** : Some of the city names are actually villages rather than municipalities or CDPs. These values are hard to analyze since they are not city names that can be as a unit of census locations. These values have to be removed from our dataset.

4. **Boroughs** : Some of the city names are actually boroughs. Since each borough has its parent town, we can use simple transformations from the boroughs to their parent town.

5. **Abbreviations or zip codes** : Our main focus of data cleaning which has to be done manually. Some of these values have their names in short (_e.g. W Haven for West Haven_) or in zip code by mistake (_e.g. 06340 for Groton, CT or Ledyard, CT_). Zip codes may stand for multiple municipalities sharing the same zip code. However, we would like to focus on deviations on County level, which means that **city values can be null but county names have to be non-null**.

6. **NaN values** : As is np.nan data type. Can be deduced from the geometries.

We can now remove the NaN values from this array to see the sorted string arrays in an alphabetical order.

In [9]:
sorted_city_names = np.sort(drug_related_deaths['DeathCity'].unique().astype(str))
sorted_city_names

array(['06340', 'AMSTON', 'ANDOVER', 'ANSONIA', 'ASHFORD', 'AVON',
       'BAKERSVILLE', 'BALTIC', 'BANTAM', 'BARKHAMSTED', 'BEACON FALLS',
       'BERLIN', 'BETHANY', 'BETHEL', 'BETHLEHEM', 'BLOOMFIELD', 'BOLTON',
       'BOZRAH', 'BRANFORD', 'BRIDGEPORT', 'BRIDGEWATER', 'BRISTOL',
       'BROAD BROOK', 'BROOKFIELD', 'BROOKLYN', 'BURLINGTON',
       'CANTERBURY', 'CANTON', 'CENTRAL VILLAGE', 'CHAPLIN', 'CHESHIRE',
       'CHESTER', 'CLINTON', 'COLCHESTER', 'COLUMBIA', 'CORNWALL BRIDGE',
       'COS COB', 'COVENTRY', 'CROMWELL', 'DANBURY', 'DANIELSON',
       'DARIEN', 'DAYVILLE', 'DEEP RIVER', 'DERBY', 'DURHAM',
       'EAST CANAAN', 'EAST GRANBY', 'EAST HADDAM', 'EAST HAMPTON',
       'EAST HARTFORD', 'EAST HARTLAND', 'EAST HAVEN', 'EAST LYME',
       'EAST WINDSOR', 'EAST WOODSTOCK', 'EASTFORD', 'EASTON',
       'ELLINGTON', 'ENFIELD', 'ESSEX', 'FAIRFIELD', 'FARMINGTON',
       'FRANKLIN', 'GALES FERRY', 'GLASTONBURY', 'GOSHEN', 'GRANBY',
       'GREENWICH', 'GRISWOLD', 'GROTON', 'G

In [10]:
res_city_not_null = drug_related_deaths[~(drug_related_deaths['ResidenceCounty'].isnull())]

res_city_not_null[res_city_not_null['ResidenceCounty'].str.contains('MIAMI')]

,ID,Date,DateType,Age,Sex,Race,ResidenceCity,ResidenceCounty,ResidenceState,DeathCity,...,Tramad,Morphine_NotHeroin,Hydromorphone,Other,OpiateNOS,AnyOpioid,MannerofDeath,DeathCityGeo,ResidenceCityGeo,InjuryCityGeo
4589,13-0424,11/23/2013 12:00:00 AM,DateofDeath,29.0,Male,White,AVENTURA,MIAMI-DADE,NaN,GREENWICH,...,NaN,NaN,NaN,ZOLP,NaN,NaN,Accident,"GREENWICH, CT\n(41.026526, -73.628549)",NaN,"CT\n(41.575155, -72.738288)"


---
### Supplementary Dataset : Municipalities in Conneticut (_mnc\_ct for short_)

Towns (light grey) and Cities (dark grey) of Connecticut

<img src="https://upload.wikimedia.org/wikipedia/commons/4/46/Towns_of_connecticut.svg" width="50%">

In [11]:
mnc_ct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169 entries, 0 to 168
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Number                   169 non-null    int64  
 1   Town                     169 non-null    object 
 2   Designation              169 non-null    object 
 3   Dateestablished          169 non-null    int64  
 4   Land area(square miles)  169 non-null    float64
 5   Population(in 2010)      169 non-null    object 
 6   Form ofgovernment        169 non-null    object 
 7   County                   169 non-null    object 
 8   Council of Governments   169 non-null    object 
 9   Native Americanname      44 non-null     object 
dtypes: float64(1), int64(2), object(7)
memory usage: 13.3+ KB


#### Brief  Observational EDA for `mnc_ct` DataFrame

As an auxilliary dataset from [Wikipedia](https://en.wikipedia.org/wiki/List_of_towns_in_Connecticut), all our focus is the reliability and whether this dataset can be used in table joins. In this dataset, the granularity is each city or town in the state of Connecticut. Null value can only be found in entries of column `Native Americanname`, which our main dataset does not cover. In this way the dataset is reliable and faithful.

In [12]:
# Extract information in columns Town and County.
mnc_ct = mnc_ct[['Town', 'County']].astype('str')
mnc_ct

,Town,County
0,Andover,Tolland County
1,Ansonia,New Haven County
2,Ashford,Windham County
3,Avon,Hartford County
4,Barkhamsted,Litchfield County
...,...,...
164,Windsor Locks,Hartford County
165,Wolcott,New Haven County
166,Woodbridge,New Haven County
167,Woodbury,Litchfield County


In [13]:
mnc_ct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169 entries, 0 to 168
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Town    169 non-null    object
 1   County  169 non-null    object
dtypes: object(2)
memory usage: 2.8+ KB


In [14]:
mnc_ct = mnc_ct.apply(lambda x: x.str.upper(), axis=1)
mnc_ct.head()

,Town,County
0,ANDOVER,TOLLAND COUNTY
1,ANSONIA,NEW HAVEN COUNTY
2,ASHFORD,WINDHAM COUNTY
3,AVON,HARTFORD COUNTY
4,BARKHAMSTED,LITCHFIELD COUNTY


In [15]:
mnc_ct['County'] = mnc_ct['County'].str.rsplit(' ', n=1, expand=True)\
                                   .loc[:, 0] # Split strings in a reverse way, only split "County"
mnc_ct.head()

,Town,County
0,ANDOVER,TOLLAND
1,ANSONIA,NEW HAVEN
2,ASHFORD,WINDHAM
3,AVON,HARTFORD
4,BARKHAMSTED,LITCHFIELD


### Supplementary Dataset : Census-designated places in Conneticut (_cdp\_ct for short_)

In [16]:
cdp_ct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CDP         112 non-null    object
 1   Population  112 non-null    object
 2   County      112 non-null    object
dtypes: object(3)
memory usage: 2.8+ KB


In [17]:
cdp_ct = cdp_ct[['CDP', 'County']]
cdp_ct.head()

,CDP,County
0,Baltic,New London
1,Bethel,Fairfield
2,Bethlehem Village,Litchfield
3,Blue Hills,Hartford
4,Branford Center,New Haven


In [18]:
cdp_ct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CDP     112 non-null    object
 1   County  112 non-null    object
dtypes: object(2)
memory usage: 1.9+ KB


In [19]:
cdp_ct = cdp_ct.apply(lambda x: x.str.upper(), axis=1)
cdp_ct.head()

,CDP,County
0,BALTIC,NEW LONDON
1,BETHEL,FAIRFIELD
2,BETHLEHEM VILLAGE,LITCHFIELD
3,BLUE HILLS,HARTFORD
4,BRANFORD CENTER,NEW HAVEN


In [20]:
all_ct = pd.concat([mnc_ct.rename({'Town': 'City'}, axis=1), cdp_ct.rename({'CDP': 'City'}, axis=1)], ignore_index=True)
all_ct

,City,County
0,ANDOVER,TOLLAND
1,ANSONIA,NEW HAVEN
2,ASHFORD,WINDHAM
3,AVON,HARTFORD
4,BARKHAMSTED,LITCHFIELD
...,...,...
276,WILLIMANTIC,WINDHAM
277,WILTON CENTER,FAIRFIELD
278,WINDSOR LOCKS,HARTFORD
279,WINSTED,LITCHFIELD


In [21]:
all_ct = all_ct.drop_duplicates().reset_index(drop=True).sort_values(['County', 'City'])
all_ct

,City,County
8,BETHEL,FAIRFIELD
14,BRIDGEPORT,FAIRFIELD
17,BROOKFIELD,FAIRFIELD
174,BYRAM,FAIRFIELD
175,CANNONDALE,FAIRFIELD
...,...,...
140,THOMPSON,WINDHAM
241,WAUREGAN,WINDHAM
245,WILLIMANTIC,WINDHAM
162,WINDHAM,WINDHAM


In [22]:
sorted_city_names[~np.isin(sorted_city_names, all_ct['City'])]

array(['06340', 'AMSTON', 'BAKERSVILLE', 'BANTAM', 'CENTRAL VILLAGE',
       'CORNWALL BRIDGE', 'DANIELSON', 'DAYVILLE', 'EAST CANAAN',
       'EAST HARTLAND', 'EAST WOODSTOCK', 'GROTON LONG POINT', 'HANOVER',
       'IVORYTON', 'JEWETT CITY', 'N HAVEN', 'NEW CANNAN', 'NO HAVEN',
       'NORTH GROSVENORDALE', 'NORTH WINDAM', 'NORTH WINDHAM',
       'NORTHFORD', 'OAKDALE', 'ONECO', 'PLANTSVILLE', 'QUAKER HILL',
       'ROGERS', 'S GLASTONBURY', 'SANDY HOOK', 'STAFFORD SPGS',
       'TAFTVILLE', 'UNCASVILLE', 'UNIONVILLE', 'VERNON-ROCKVILLE',
       'W HAVEN', 'WEST STAFFORD', 'WEST SUFFIELD', 'nan'], dtype='<U19')

### Auxiliary Dataset : Boroughs in Conneticut (_br\_ct for short_)

In [23]:
br_ct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Borough            45 non-null     object
 1   Parent town        45 non-null     object
 2   Date incorporated  45 non-null     object
 3   Current status     45 non-null     object
dtypes: object(4)
memory usage: 1.5+ KB


In [24]:
br_ct.query("`Current status` == 'Borough'")

,Borough,Parent town,Date incorporated,Current status
1,Stonington,Stonington,1801,Borough
6,Newtown,Newtown,1824,Borough
21,Danielson,Killingly,1854,Borough
30,Litchfield,Litchfield,1879,Borough
37,Jewett City,Griswold,1895,Borough
38,Fenwick,Old Saybrook,1899,Borough
42,Woodmont,Milford,1903,Borough
43,Bantam,Litchfield,1915,Borough


In [25]:
br_to_town = br_ct.query("`Current status` == 'Borough'")\
                  .apply(lambda x: x.str.upper(), axis=1)\
                  .set_index('Borough')\
                  .loc[:, 'Parent town']\
                  .to_dict()
br_to_town

{'STONINGTON': 'STONINGTON',
 'NEWTOWN': 'NEWTOWN',
 'DANIELSON': 'KILLINGLY',
 'LITCHFIELD': 'LITCHFIELD',
 'JEWETT CITY': 'GRISWOLD',
 'FENWICK': 'OLD SAYBROOK',
 'WOODMONT': 'MILFORD',
 'BANTAM': 'LITCHFIELD'}

### Manual dictionary processors of city names

In [26]:
manual_rem = sorted_city_names[(~np.isin(sorted_city_names, all_ct['City'])) & (~np.isin(sorted_city_names, list(br_to_town.keys())))]
manual_rem

array(['06340', 'AMSTON', 'BAKERSVILLE', 'CENTRAL VILLAGE',
       'CORNWALL BRIDGE', 'DAYVILLE', 'EAST CANAAN', 'EAST HARTLAND',
       'EAST WOODSTOCK', 'GROTON LONG POINT', 'HANOVER', 'IVORYTON',
       'N HAVEN', 'NEW CANNAN', 'NO HAVEN', 'NORTH GROSVENORDALE',
       'NORTH WINDAM', 'NORTH WINDHAM', 'NORTHFORD', 'OAKDALE', 'ONECO',
       'PLANTSVILLE', 'QUAKER HILL', 'ROGERS', 'S GLASTONBURY',
       'SANDY HOOK', 'STAFFORD SPGS', 'TAFTVILLE', 'UNCASVILLE',
       'UNIONVILLE', 'VERNON-ROCKVILLE', 'W HAVEN', 'WEST STAFFORD',
       'WEST SUFFIELD', 'nan'], dtype='<U19')

In [27]:
manual_rem = pd.Series(manual_rem)
dir_rem = manual_rem[manual_rem.str.contains(r'(^NORTH|WEST|EAST|SOUTH)\s')]
dir_rem

/home/muhan-zhang/miniconda3/envs/msse-python/lib/python3.7/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


6             EAST CANAAN
7           EAST HARTLAND
8          EAST WOODSTOCK
15    NORTH GROSVENORDALE
16           NORTH WINDAM
17          NORTH WINDHAM
32          WEST STAFFORD
33          WEST SUFFIELD
dtype: object

In [28]:
to_no_dir = {i: j for i, j in zip(dir_rem, dir_rem.str.split(' ', expand=True)[1])}

# Manual changes
to_no_dir['NORTH WINDAM'] = 'WINDHAM'
to_no_dir['NORTH GROSVENORDALE'] = 'NORTH GROSVENOR DALE'

to_no_dir

{'EAST CANAAN': 'CANAAN',
 'EAST HARTLAND': 'HARTLAND',
 'EAST WOODSTOCK': 'WOODSTOCK',
 'NORTH GROSVENORDALE': 'NORTH GROSVENOR DALE',
 'NORTH WINDAM': 'WINDHAM',
 'NORTH WINDHAM': 'WINDHAM',
 'WEST STAFFORD': 'STAFFORD',
 'WEST SUFFIELD': 'SUFFIELD'}

In [29]:
from_abbr_zip = {
    '06340': 'GROTON',
    'N HAVEN': 'NORTH HAVEN',
    'NEW CANNAN': 'NEW CANAAN',
    'NO HAVEN': 'NORTH HAVEN',
    'W HAVEN': 'WEST HAVEN',
    'S GLASTONBURY': 'GLASTONBURY CENTER',
    'VERNON-ROCKVILLE': 'ROCKVILLE',
    'STAFFORD SPGS': 'STAFFORD SPRINGS',
    'GROTON LONG POINT': 'GROTON'
}

In [30]:
to_city_names = {**br_to_town, **to_no_dir, **from_abbr_zip}
to_city_names

{'STONINGTON': 'STONINGTON',
 'NEWTOWN': 'NEWTOWN',
 'DANIELSON': 'KILLINGLY',
 'LITCHFIELD': 'LITCHFIELD',
 'JEWETT CITY': 'GRISWOLD',
 'FENWICK': 'OLD SAYBROOK',
 'WOODMONT': 'MILFORD',
 'BANTAM': 'LITCHFIELD',
 'EAST CANAAN': 'CANAAN',
 'EAST HARTLAND': 'HARTLAND',
 'EAST WOODSTOCK': 'WOODSTOCK',
 'NORTH GROSVENORDALE': 'NORTH GROSVENOR DALE',
 'NORTH WINDAM': 'WINDHAM',
 'NORTH WINDHAM': 'WINDHAM',
 'WEST STAFFORD': 'STAFFORD',
 'WEST SUFFIELD': 'SUFFIELD',
 '06340': 'GROTON',
 'N HAVEN': 'NORTH HAVEN',
 'NEW CANNAN': 'NEW CANAAN',
 'NO HAVEN': 'NORTH HAVEN',
 'W HAVEN': 'WEST HAVEN',
 'S GLASTONBURY': 'GLASTONBURY CENTER',
 'VERNON-ROCKVILLE': 'ROCKVILLE',
 'STAFFORD SPGS': 'STAFFORD SPRINGS',
 'GROTON LONG POINT': 'GROTON'}

## Step 2 : Data Cleaning

### 2.1 Death city cleaning and county fillna

In [31]:
drug_related_deaths.head()

,ID,Date,DateType,Age,Sex,Race,ResidenceCity,ResidenceCounty,ResidenceState,DeathCity,...,Tramad,Morphine_NotHeroin,Hydromorphone,Other,OpiateNOS,AnyOpioid,MannerofDeath,DeathCityGeo,ResidenceCityGeo,InjuryCityGeo
0,14-0273,06/28/2014 12:00:00 AM,DateReported,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Accident,"CT\n(41.575155, -72.738288)","CT\n(41.575155, -72.738288)","CT\n(41.575155, -72.738288)"
1,13-0102,03/21/2013 12:00:00 AM,DateofDeath,48.0,Male,Black,NORWALK,NaN,NaN,NORWALK,...,NaN,NaN,NaN,NaN,NaN,NaN,Accident,"Norwalk, CT\n(41.11805, -73.412906)","NORWALK, CT\n(41.11805, -73.412906)","CT\n(41.575155, -72.738288)"
2,16-0165,03/13/2016 12:00:00 AM,DateofDeath,30.0,Female,White,SANDY HOOK,FAIRFIELD,CT,DANBURY,...,NaN,NaN,NaN,NaN,NaN,Y,Accident,"Danbury, CT\n(41.393666, -73.451539)","SANDY HOOK, CT\n(41.419998, -73.282501)",NaN
3,16-0208,03/31/2016 12:00:00 AM,DateofDeath,23.0,Male,White,RYE,WESTCHESTER,NY,GREENWICH,...,NaN,NaN,NaN,NaN,NaN,Y,Accident,"Greenwich, CT\n(41.026526, -73.628549)",NaN,NaN
4,13-0052,02/13/2013 12:00:00 AM,DateofDeath,22.0,Male,"Asian, Other",FLUSHING,QUEENS,NaN,GREENWICH,...,NaN,NaN,NaN,NaN,NaN,NaN,Accident,"Greenwich, CT\n(41.026526, -73.628549)",NaN,"CT\n(41.575155, -72.738288)"


In [32]:
drug_related_deaths.loc[drug_related_deaths['DeathCity'].isin(list(to_city_names.keys())), 'DeathCity']\
    = drug_related_deaths.loc[drug_related_deaths['DeathCity'].isin(list(to_city_names.keys())), 'DeathCity']\
                         .map(to_city_names, na_action='ignore')
sorted_city_names_cleaned = np.sort(drug_related_deaths['DeathCity'].unique().astype(str))
rem_to_ignore = sorted_city_names_cleaned[(~np.isin(sorted_city_names_cleaned, all_ct['City'])) \
                                         & (~np.isin(sorted_city_names_cleaned, 'nan'))]
rem_to_ignore

array(['AMSTON', 'BAKERSVILLE', 'CENTRAL VILLAGE', 'CORNWALL BRIDGE',
       'DAYVILLE', 'HANOVER', 'IVORYTON', 'NORTHFORD', 'OAKDALE', 'ONECO',
       'PLANTSVILLE', 'QUAKER HILL', 'ROGERS', 'SANDY HOOK', 'TAFTVILLE',
       'UNCASVILLE', 'UNIONVILLE'], dtype='<U20')

In [33]:
drug_related_deaths_city_cleaned = drug_related_deaths.dropna(subset=['DeathCity'])\
                                                      .loc[~(drug_related_deaths['DeathCity'].isin(rem_to_ignore)), :]
drug_related_deaths_city_cleaned

,ID,Date,DateType,Age,Sex,Race,ResidenceCity,ResidenceCounty,ResidenceState,DeathCity,...,Tramad,Morphine_NotHeroin,Hydromorphone,Other,OpiateNOS,AnyOpioid,MannerofDeath,DeathCityGeo,ResidenceCityGeo,InjuryCityGeo
1,13-0102,03/21/2013 12:00:00 AM,DateofDeath,48.0,Male,Black,NORWALK,NaN,NaN,NORWALK,...,NaN,NaN,NaN,NaN,NaN,NaN,Accident,"Norwalk, CT\n(41.11805, -73.412906)","NORWALK, CT\n(41.11805, -73.412906)","CT\n(41.575155, -72.738288)"
2,16-0165,03/13/2016 12:00:00 AM,DateofDeath,30.0,Female,White,SANDY HOOK,FAIRFIELD,CT,DANBURY,...,NaN,NaN,NaN,NaN,NaN,Y,Accident,"Danbury, CT\n(41.393666, -73.451539)","SANDY HOOK, CT\n(41.419998, -73.282501)",NaN
3,16-0208,03/31/2016 12:00:00 AM,DateofDeath,23.0,Male,White,RYE,WESTCHESTER,NY,GREENWICH,...,NaN,NaN,NaN,NaN,NaN,Y,Accident,"Greenwich, CT\n(41.026526, -73.628549)",NaN,NaN
4,13-0052,02/13/2013 12:00:00 AM,DateofDeath,22.0,Male,"Asian, Other",FLUSHING,QUEENS,NaN,GREENWICH,...,NaN,NaN,NaN,NaN,NaN,NaN,Accident,"Greenwich, CT\n(41.026526, -73.628549)",NaN,"CT\n(41.575155, -72.738288)"
5,14-0277,06/29/2014 12:00:00 AM,DateofDeath,23.0,Male,White,BRISTOL,NaN,NaN,BRISTOL,...,NaN,NaN,NaN,NaN,NaN,NaN,Accident,"BRISTOL, CT\n(41.673037, -72.945791)","BRISTOL, CT\n(41.673037, -72.945791)","CT\n(41.575155, -72.738288)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5100,15-0466,09/08/2015 12:00:00 AM,DateReported,43.0,Male,White,CHESHIRE,NEW HAVEN,CT,CHESHIRE,...,NaN,NaN,NaN,NaN,NaN,NaN,Accident,"CHESHIRE, CT\n(41.498834, -72.901448)","CHESHIRE, CT\n(41.498834, -72.901448)","CT\n(41.575155, -72.738288)"
5101,17-0618,07/22/2017 12:00:00 AM,DateReported,21.0,Male,White,MADISON,NEW HAVEN,CT,NEW HAVEN,...,NaN,NaN,NaN,NaN,NaN,NaN,Accident,"New Haven, CT\n(41.308252, -72.924161)","MADISON, CT\n(41.271447, -72.60086)","CT\n(41.575155, -72.738288)"
5102,18-0646,08/14/2018 12:00:00 AM,DateofDeath,30.0,Male,White,LAWRENCEVILLE,TIOGA,PA,DANBURY,...,Y,NaN,NaN,NaN,NaN,Y,Accident,"DANBURY, CT\n(41.393666, -73.451539)",NaN,"DANBURY, CT\n(41.393666, -73.451539)"
5103,14-0124,03/16/2014 12:00:00 AM,DateofDeath,33.0,Male,White,HARTFORD,NaN,NaN,WINDSOR,...,NaN,NaN,NaN,NaN,NaN,NaN,Accident,"WINDSOR, CT\n(41.852781, -72.64379)","HARTFORD, CT\n(41.765775, -72.673356)","CT\n(41.575155, -72.738288)"


In [34]:
drug_related_deaths_cleaned = pd.merge(drug_related_deaths_city_cleaned, all_ct, how='left', left_on='DeathCity', right_on='City')
drug_related_deaths_cleaned.head()

,ID,Date,DateType,Age,Sex,Race,ResidenceCity,ResidenceCounty,ResidenceState,DeathCity,...,Hydromorphone,Other,OpiateNOS,AnyOpioid,MannerofDeath,DeathCityGeo,ResidenceCityGeo,InjuryCityGeo,City,County
0,13-0102,03/21/2013 12:00:00 AM,DateofDeath,48.0,Male,Black,NORWALK,NaN,NaN,NORWALK,...,NaN,NaN,NaN,NaN,Accident,"Norwalk, CT\n(41.11805, -73.412906)","NORWALK, CT\n(41.11805, -73.412906)","CT\n(41.575155, -72.738288)",NORWALK,FAIRFIELD
1,16-0165,03/13/2016 12:00:00 AM,DateofDeath,30.0,Female,White,SANDY HOOK,FAIRFIELD,CT,DANBURY,...,NaN,NaN,NaN,Y,Accident,"Danbury, CT\n(41.393666, -73.451539)","SANDY HOOK, CT\n(41.419998, -73.282501)",NaN,DANBURY,FAIRFIELD
2,16-0208,03/31/2016 12:00:00 AM,DateofDeath,23.0,Male,White,RYE,WESTCHESTER,NY,GREENWICH,...,NaN,NaN,NaN,Y,Accident,"Greenwich, CT\n(41.026526, -73.628549)",NaN,NaN,GREENWICH,FAIRFIELD
3,13-0052,02/13/2013 12:00:00 AM,DateofDeath,22.0,Male,"Asian, Other",FLUSHING,QUEENS,NaN,GREENWICH,...,NaN,NaN,NaN,NaN,Accident,"Greenwich, CT\n(41.026526, -73.628549)",NaN,"CT\n(41.575155, -72.738288)",GREENWICH,FAIRFIELD
4,14-0277,06/29/2014 12:00:00 AM,DateofDeath,23.0,Male,White,BRISTOL,NaN,NaN,BRISTOL,...,NaN,NaN,NaN,NaN,Accident,"BRISTOL, CT\n(41.673037, -72.945791)","BRISTOL, CT\n(41.673037, -72.945791)","CT\n(41.575155, -72.738288)",BRISTOL,HARTFORD


In [35]:
check_counties = drug_related_deaths_cleaned[['DeathCounty', 'County']].dropna(subset=['DeathCounty'])
check_counties

,DeathCounty,County
0,FAIRFIELD,FAIRFIELD
3,FAIRFIELD,FAIRFIELD
4,HARTFORD,HARTFORD
5,HARTFORD,HARTFORD
6,NEW HAVEN,NEW HAVEN
...,...,...
5044,LITCHFIELD,LITCHFIELD
5045,NEW HAVEN,NEW HAVEN
5046,NEW HAVEN,NEW HAVEN
5047,NEW HAVEN,NEW HAVEN


In [36]:
drug_related_deaths_cleaned.loc[check_counties[check_counties['DeathCounty'] != check_counties['County']].index]

,ID,Date,DateType,Age,Sex,Race,ResidenceCity,ResidenceCounty,ResidenceState,DeathCity,...,Hydromorphone,Other,OpiateNOS,AnyOpioid,MannerofDeath,DeathCityGeo,ResidenceCityGeo,InjuryCityGeo,City,County
2049,13-0339,10/07/2013 12:00:00 AM,DateofDeath,56.0,Female,White,HARTFORD,USA,NaN,HARTFORD,...,NaN,NaN,NaN,NaN,Accident,"Hartford, CT\n(41.765775, -72.673356)","HARTFORD, CT\n(41.765775, -72.673356)","CT\n(41.575155, -72.738288)",HARTFORD,HARTFORD


In [37]:
drug_related_deaths_cleaned['DeathCounty'] = drug_related_deaths_cleaned['County']
drug_related_deaths_cleaned = drug_related_deaths_cleaned.drop(['City', 'County'], axis=1)
drug_related_deaths_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5051 entries, 0 to 5050
Data columns (total 41 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   5051 non-null   object 
 1   Date                 5051 non-null   object 
 2   DateType             5051 non-null   object 
 3   Age                  5051 non-null   float64
 4   Sex                  5048 non-null   object 
 5   Race                 5041 non-null   object 
 6   ResidenceCity        4882 non-null   object 
 7   ResidenceCounty      4262 non-null   object 
 8   ResidenceState       3518 non-null   object 
 9   DeathCity            5051 non-null   object 
 10  DeathCounty          5051 non-null   object 
 11  Location             5031 non-null   object 
 12  LocationifOther      582 non-null    object 
 13  DescriptionofInjury  4282 non-null   object 
 14  InjuryPlace          4986 non-null   object 
 15  InjuryCity           3313 non-null   o

### 2.2 Geometry (Latitude, Logitude) separation

In [38]:
def separate_latitude_longitude(col_name):
    geo = drug_related_deaths_cleaned[col_name].str.replace('\n', '')\
                                               .str.replace(r'^.*\(([-\d.]+), ([-\d.]+)\)$',
                                                            lambda x: x.group(1) + ',' + x.group(2))\
                                               .str.split(',', expand=True)\
                                               .astype(float)\
                                               .rename({0: col_name + 'Lat', 1: col_name + 'Log'}, axis=1)
    return geo

In [39]:
geos_to_change = [i + 'CityGeo' for i in ['Death', 'Residence', 'Injury']]
geos = [separate_latitude_longitude(i) for i in geos_to_change]

In [40]:
drug_related_deaths_geo = pd.concat([drug_related_deaths_cleaned] + geos, axis=1).drop(geos_to_change, axis=1)
drug_related_deaths_geo.head()

,ID,Date,DateType,Age,Sex,Race,ResidenceCity,ResidenceCounty,ResidenceState,DeathCity,...,Other,OpiateNOS,AnyOpioid,MannerofDeath,DeathCityGeoLat,DeathCityGeoLog,ResidenceCityGeoLat,ResidenceCityGeoLog,InjuryCityGeoLat,InjuryCityGeoLog
0,13-0102,03/21/2013 12:00:00 AM,DateofDeath,48.0,Male,Black,NORWALK,NaN,NaN,NORWALK,...,NaN,NaN,NaN,Accident,41.118050,-73.412906,41.118050,-73.412906,41.575155,-72.738288
1,16-0165,03/13/2016 12:00:00 AM,DateofDeath,30.0,Female,White,SANDY HOOK,FAIRFIELD,CT,DANBURY,...,NaN,NaN,Y,Accident,41.393666,-73.451539,41.419998,-73.282501,NaN,NaN
2,16-0208,03/31/2016 12:00:00 AM,DateofDeath,23.0,Male,White,RYE,WESTCHESTER,NY,GREENWICH,...,NaN,NaN,Y,Accident,41.026526,-73.628549,NaN,NaN,NaN,NaN
3,13-0052,02/13/2013 12:00:00 AM,DateofDeath,22.0,Male,"Asian, Other",FLUSHING,QUEENS,NaN,GREENWICH,...,NaN,NaN,NaN,Accident,41.026526,-73.628549,NaN,NaN,41.575155,-72.738288
4,14-0277,06/29/2014 12:00:00 AM,DateofDeath,23.0,Male,White,BRISTOL,NaN,NaN,BRISTOL,...,NaN,NaN,NaN,Accident,41.673037,-72.945791,41.673037,-72.945791,41.575155,-72.738288


### 2.3 Date transformation

In [41]:
drug_related_deaths_geo['Date'] = drug_related_deaths_geo['Date'].astype('datetime64[ns]')

In [42]:
drug_related_deaths_geo.dtypes

ID                             object
Date                   datetime64[ns]
DateType                       object
Age                           float64
Sex                            object
                            ...      
DeathCityGeoLog               float64
ResidenceCityGeoLat           float64
ResidenceCityGeoLog           float64
InjuryCityGeoLat              float64
InjuryCityGeoLog              float64
Length: 44, dtype: object

In [43]:
times = drug_related_deaths_geo['Date'].dt.time
times.head()

0    00:00:00
1    00:00:00
2    00:00:00
3    00:00:00
4    00:00:00
Name: Date, dtype: object

In [44]:
times.unique()

array([datetime.time(0, 0)], dtype=object)

This implies the time of the day only shares one same value: 12:00 AM, on which we cannot rely. Only the day of the week or the year can be used as variables to analyze.

### 2.4 One-hot encoding of drug classifications

In [45]:
drug_related_deaths_geo = drug_related_deaths_geo.drop('Other', axis=1)
drug_related_deaths_geo.loc[:, 'Heroin': 'AnyOpioid']

,Heroin,Cocaine,Fentanyl,FentanylAnalogue,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphet,Tramad,Morphine_NotHeroin,Hydromorphone,OpiateNOS,AnyOpioid
0,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
2,Y,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
3,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5046,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5047,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5048,Y,NaN,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,Y
5049,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
drug_classifications = drug_related_deaths_geo.loc[:, 'Heroin': 'AnyOpioid']
drug_classifications.head()

,Heroin,Cocaine,Fentanyl,FentanylAnalogue,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphet,Tramad,Morphine_NotHeroin,Hydromorphone,OpiateNOS,AnyOpioid
0,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
2,Y,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
3,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
drug_classifications = drug_classifications.replace({'Y': 1, np.nan: 0})
drug_classifications[~((drug_classifications == 0) | (drug_classifications == 1)).all(axis=1)]

,Heroin,Cocaine,Fentanyl,FentanylAnalogue,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphet,Tramad,Morphine_NotHeroin,Hydromorphone,OpiateNOS,AnyOpioid
60,0,0,0,0,0,0,0,0,1,0,0,0,YES,0,0,1
261,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,N
451,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,N
502,1,1,Y-A,0,0,0,0,0,0,0,0,0,0,0,0,1
578,0,0,1,0,0,0,0,0,1,0,0,0,STOLE MEDS,0,1,1
2780,0,0,Y POPS,0,1,0,0,0,1,0,0,0,0,0,0,0
3181,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,N
3464,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,N
3703,0,0,Y (PTCH),0,0,0,0,0,1,0,0,0,0,0,0,0
3761,1,0,Y-A,0,0,0,0,0,1,0,1,0,0,0,0,1


#### Data cleaning of drug classifications

From the DataFrame above, mismatches of data type (int) occur to some values. Here represent what are deductible from these mismatching values:

1. `Y-A`, `Y POPS`, `Y (PTCH)`, `YES`: All stand for `Y`, which refers to OHE value 1. All contents after hyphen or space are classifications in detail under corresponding drug subset.
2. `STOLE MEDS`, `NO RX BUT STRAWS`:

> The `Morphine (Not Heroin)` values are related to the differences between how Morphine and Heroin are metabolized and therefore detected in the toxicity results. Heroin metabolizes to 6-MAM which then metabolizes to morphine. 6-MAM is unique to heroin and has a short half-life (as does heroin itself). Thus, in some heroin deaths, the toxicity results will not indicate whether the morphine is from heroin or prescription morphine. In these cases the Medical Examiner may be able to determine the cause based on the scene investigation (such as finding heroin needles). If they find prescription morphine at the scene it is certified as “Morphine (not heroin).” Therefore, the Cause of Death may indicate Morphine, but the Heroin or Morphine (Not Heroin) may not be indicated. (From [Dataset codebook](https://www.kaggle.com/muhakabartay/accidental-drug-related-deaths-20122018))

In this way, these two values both stand for `Y`, which refers to OHE value 1.

3. `PCP NEG`: This stands for "Phencyclidine or phenylcyclohexyl piperidine Negative". In this way, this value refer to OHE value 1.

4. `N` : Obviously refer to OHE value 0.

In [48]:
drug_classifications = drug_classifications.replace({'Y': 1, 
                                                     np.nan: 0,
                                                     'Y-A': 1,
                                                     'Y POPS': 1,
                                                     'Y (PTCH)': 1,
                                                     'YES': 1,
                                                     'STOLE MEDS': 1,
                                                     'NO RX BUT STRAWS': 1,
                                                     'PCP NEG': 0,
                                                     'N': 0})
drug_classifications[~((drug_classifications == 0) | (drug_classifications == 1)).all(axis=1)]

,Heroin,Cocaine,Fentanyl,FentanylAnalogue,Oxycodone,Oxymorphone,Ethanol,Hydrocodone,Benzodiazepine,Methadone,Amphet,Tramad,Morphine_NotHeroin,Hydromorphone,OpiateNOS,AnyOpioid


In [49]:
np.linalg.matrix_rank(drug_classifications.values.astype(int))

16

## Step 3 : Visualization

In [43]:
CT_COORDINATES = (41.5, -72.8)
ct_map = folium.Map(location=CT_COORDINATES, zoom_start=9)
locs = drug_related_deaths_geo[['DeathCityGeoLat', 'DeathCityGeoLog']].astype('float').dropna().to_numpy()
heatmap = folium.plugins.HeatMap(locs.tolist(), radius = 25)
ct_map.add_child(heatmap)

---
# VISUALIZATIONS HAVE TO BE WRITTEN ABOVE